In [1]:
import GMesh
import math
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read topo data
!cp -n /archive/gold/datasets/topography/GEBCO_08_v2.nc .
with netCDF4.Dataset('GEBCO_08_v2.nc') as nc:
    topo_lon = nc.variables['longitude'][:].filled(0.)
    topo_lat = nc.variables['latitude'][:].filled(0.)
    topo_depth = nc.variables['depth'][:,:].filled(0.)
#! cp -n /archive/gold/datasets/topography/GEBCO_2023/GEBCO_2023.nc .
# with netCDF4.Dataset('/archive/gold/datasets/topography/GEBCO_2023/GEBCO_2023.nc') as nc:
#     topo_lon = nc.variables['lon'][:].filled(0.)
#     topo_lat = nc.variables['lat'][:].filled(0.)
#     topo_depth = nc.variables['elevation'][:,:].filled(0.)

In [3]:
print(topo_lon.min(), topo_lon.max(), topo_lat.min(), topo_lat.max())

-179.99583333333334 179.99583333333334 -89.99583333333334 89.99583333333334


In [4]:
def convol( levels, h, f, verbose=False ):
    """Coarsens the product of h*f across all levels"""
    levels[-1].height = ( h * f ).reshape(levels[-1].nj,levels[-1].ni)
    for k in range( len(levels) - 1, 0, -1 ):
        if verbose: print('Coarsening {} -> {}'.format(k,k-1))
        levels[k].coarsenby2( levels[k-1] )
    return levels[0].height

In [5]:
def rough( levels, h, h2min=1.e-7 ):
    """Calculates both mean of H, and variance of H relative to a plane"""
    # Construction weights for momenet calculations
    nx = 2**( len(levels) - 1 )
    x = ( np.arange(nx) - ( nx - 1 ) /2 ) * np.sqrt( 12 / ( nx**2 - 1 ) )
    X, Y = np.meshgrid( x, x )
    X, Y = X.reshape(1,nx,1,nx), Y.reshape(1,nx,1,nx)
    h = h.reshape(levels[0].nj,nx,levels[0].ni,nx)
    H2 = convol( levels, h, h )
    HX = convol( levels, h, X )
    HY = convol( levels, h, Y )
    H = convol( levels, h, np.ones((1,nx,1,nx)) )
    return H, H2 - H**2 - HX**2 - HY**2 + h2min

In [6]:
import importlib
importlib.reload(GMesh)

<module 'GMesh' from '/local2/home/GitHub/thin-wall-topography/python/GMesh.py'>

In [7]:
# Read target mesh
with netCDF4.Dataset('/local2/home/GitHub/OM4_025-grid-data/ocean_hgrid.nc') as nc:
    fullG = GMesh.GMesh( lon=nc.variables['x'][::2,::2], lat=nc.variables['y'][::2,::2] )

In [8]:
di, dj = fullG.ni // 3, fullG.nj // 45
print('window size dj,di =',dj,di,'full model nj,ni=',fullG.nj, fullG.ni)
Hcnt = np.zeros((fullG.nj, fullG.ni)) # Diagnostic: counting which cells we are working on
Htarg, H2targ = np.zeros((fullG.nj, fullG.ni)), np.zeros((fullG.nj, fullG.ni))
gtic = GMesh.GMesh._toc(None,"")
for j in range( fullG.nj // dj ): # 13
    csj, sj = slice( j*dj, (j+1)*dj ), slice( j*dj, (j+1)*dj+1 )
    print('sj=',sj,'csj=',csj)
    for i in range( fullG.ni // di ): # 4
        csi, si = slice( i*di, (i+1)*di ), slice( i*di, (i+1)*di+1 )
        print('si=',si,'csi=',csi)
        Hcnt[csj,csi] = Hcnt[csj,csi] + 1 # Diagnostic: counting which cells we are working on
        G = GMesh.GMesh( lon = fullG.lon[sj,si], lat = fullG.lat[sj,si] )
        print(j, i, G, G.lon.min(), G.lon.max(), G.lat.min(), G.lat.max())
        # This subsets the source topography so we don't work with too large arrays
        tslices = G.find_source_spanning_slices( topo_lon, topo_lat )
        print('topo slices =', tslices)
        # This recusrively refine the mesh until some criteria is met ...
        levels = G.refine_loop( topo_lon[tslices[0]], topo_lat[tslices[1]], resolution_limit=False, fixed_refine_level=5 )
        # Use nearest neighbor topography to populate the finest grid
        levels[-1].project_source_data_onto_target_mesh( topo_lon[tslices[0]], topo_lat[tslices[1]], topo_depth[tslices[1],tslices[0]] )
        # Now recursively coarsen
        h, h2 = rough( levels, levels[-1].height )
        # Store window in final array
        Htarg[csj,csi] = h
        H2targ[csj,csi] = h2
GMesh.GMesh._toc(gtic,"Whole workflow")
print( Hcnt.min(), Hcnt.max(), '<-- should both be 1 for full model' )

window size dj,di = 24 480 full model nj,ni= 1080 1440
sj= slice(0, 25, None) csj= slice(0, 24, None)
si= slice(0, 481, None) csi= slice(0, 480, None)
0 0 <GMesh nj:24 ni:480 shape:(24,480)> -299.99458164885345 -180.02607218723773 -80.91667601006795 -78.34532374100719
topo slices = (slice(28801, 43198, None), slice(1089, 1398, None))
Refine level 0 <GMesh nj:24 ni:480 shape:(24,480)> (0.1758 Mb)
Refine level 1 <GMesh nj:48 ni:960 shape:(48,960)> (0.7031 Mb)
Refine level 2 <GMesh nj:96 ni:1920 shape:(96,1920)> (2.8125 Mb)
Refine level 3 <GMesh nj:192 ni:3840 shape:(192,3840)> (11.2500 Mb)
Refine level 4 <GMesh nj:384 ni:7680 shape:(384,7680)> (45.0000 Mb)
Refine level 5 <GMesh nj:768 ni:15360 shape:(768,15360)> (180.0000 Mb)
si= slice(480, 961, None) csi= slice(480, 960, None)
0 1 <GMesh nj:24 ni:480 shape:(24,480)> -180.79584645537295 -59.36279393959914 -81.66101694915254 -78.59233025247781
topo slices = (slice(0, 43198, None), slice(1000, 1368, None))
Refine level 0 <GMesh nj:24 ni:48

In [ ]:
# ax = plt.subplot(111)
# G.plot(ax)

In [ ]:
levels = G.refine_loop( topo_lon[tslices[0]], topo_lat[tslices[1]], resolution_limit=False, fixed_refine_level=5, timers=True )

In [ ]:
# Use nearest neighbor topography to populate the finest grid
levels[-1].project_source_data_onto_target_mesh( topo_lon[tslices[0]], topo_lat[tslices[1]], topo_depth[tslices[1],tslices[0]], timers=True )
h = levels[-1].height # Keep copy

In [ ]:
nx = 2**( len(levels) - 1 )
x = ( np.arange(nx) - ( nx - 1 ) /2 ) * np.sqrt( 12 / ( nx**2 - 1 ) )
print('mean(x) =',x.mean(), 'mean(x^2) =',(x*x).mean())
X, Y = np.meshgrid( x, x )
X, Y = X.reshape(1,nx,1,nx), Y.reshape(1,nx,1,nx)

In [ ]:
# Now recursively coarsen for square
# h = h.reshape(levels[0].nj,nx,levels[0].ni,nx)
# H2 = convol( levels, h, h )
# HX = convol( levels, h, X )
# HY = convol( levels, h, Y )
# H = convol( levels, h, np.ones((1,nx,1,nx)) )
# h2 = H2 - H**2 - HX**2 - HY**2

In [ ]:
H, H2 = rough( levels, h )

In [ ]:
G = levels[0]
plt.pcolormesh( G.lon, G.lat, np.log10(h2), vmin=-3, vmax=6 ); plt.colorbar();

In [ ]:
# Read topo data
with netCDF4.Dataset('/archive/gold/datasets/OM4_025/mosaic.v20170622.unpacked/ocean_topog.nc') as nc:
    print(nc)
    OM4_depth = nc.variables['depth'][:,:].filled(0.)
    OM4_h2 = nc.variables['h2'][:,:].filled(0.)

In [ ]:
plt.pcolormesh( G.lon, G.lat, np.log10( OM4_h2[csj,csi] ), vmin=-3, vmax=6 ); plt.colorbar();

In [ ]:
plt.pcolormesh( G.lon, G.lat, h2 - OM4_h2[csj,csi] ); plt.colorbar();

In [ ]:
# H7, H27 = 1*H, 1*H2
# plt.pcolormesh( G.lon, G.lat, H - H7 ); plt.colorbar();

In [9]:
with netCDF4.Dataset('mean_h2_topo_3x45.nc','w','clobber') as nc:
    nx = nc.createDimension('nx', Htarg.shape[1])
    ny = nc.createDimension('ny', Htarg.shape[0])
    ntiles = nc.createDimension('ntiles', 1)
    z = nc.createVariable('depth', float, ('ny','nx') )
    z2 = nc.createVariable('h2', float, ('ny','nx') )
    z[:,:] = -Htarg[:,:]
    z2[:,:] = H2targ[:,:]